In [18]:
import pandas as pd
from datetime import datetime
import json

import warnings
warnings.filterwarnings("ignore")

fase = '06_evaluar_prediccion'

In [19]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [20]:
folder = gen_config['folder']

path_prod_data = gen_config['path_prod_data']
path_pred_test = gen_config['path_pred_test']
path_pred_futuro = gen_config['path_pred_futuro']

path_kaggle = gen_config['path_kaggle']

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
---------------------------------------06_evaluar_prediccion----------------------------------------


In [21]:
def perdida(real, prediccion):
    total = real.sum()
    perdida =  abs(real - prediccion).sum()
    resultado = perdida / total
    return resultado

In [22]:
prod_data = pd.read_parquet(f"{folder}/{path_prod_data}")
prod_a_predecir = pd.read_csv('../productos_a_predecir.txt', sep='\t')
df_test = pd.read_parquet(f"{folder}/{path_pred_test}")
df_fut = pd.read_parquet(f"{folder}/{path_pred_futuro}")

In [23]:
df_pred_test = df_test[['product_id', 'tn_futuro_real', 'tn_prediccion_real']].groupby('product_id', as_index=False).sum()
df_pred_test['error'] = -(df_pred_test['tn_futuro_real'] - df_pred_test['tn_prediccion_real'])/df_pred_test['tn_futuro_real']*100
df_pred_test['error_abs'] = -(df_pred_test['tn_futuro_real'] - df_pred_test['tn_prediccion_real'])
df_pred_test['weight'] = df_pred_test['tn_futuro_real']/df_pred_test['tn_futuro_real'].sum()
df_pred_test.sort_values('weight', ascending=False, inplace=True)

perdida = perdida(df_pred_test['tn_futuro_real'], df_pred_test['tn_prediccion_real'])
print(f"Perdida en test: {perdida}")
print(f"Primeras filas de test:\n{df_pred_test.iloc[:11,:4]}")

Perdida en test: 0.25224935811274596
Primeras filas de test:
    product_id  tn_futuro_real  tn_prediccion_real      error
0        20001      1286.02350         1203.294161  -6.432957
1        20002      1002.49082          901.328331 -10.091114
2        20003       680.22322          669.511571  -1.574726
3        20004       506.69680          485.416755  -4.199759
4        20005       491.74883          534.301940   8.653424
31       20032       402.43544          427.442425   6.213912
8        20009       360.53386          357.994773  -0.704258
5        20006       315.78911          333.460112   5.595824
18       20019       287.41511          266.773794  -7.181709
10       20011       241.39206          306.667983  27.041454
6        20007       231.85925          339.543112  46.443634


In [24]:
# df_pred_ungroup = df_pred_test[['group_index', 'tn_prediccion_real']]
# df_pred_ungroup = df_pred_ungroup.merge(prod_data_ungroup, on='group_index', how='left')
# df_pred_ungroup = df_pred_ungroup[['group_index', 'product_id', 'tn_prediccion_real', 'proporcion']]

# df_pred_ungroup = df_pred_ungroup[df_pred_ungroup['product_id'].isin(prod_a_predecir['product_id'])]
# df_pred_ungroup['prediccion_final'] = df_pred_ungroup['tn_prediccion_real'] * df_pred_ungroup['proporcion']
# df_pred_ungroup.sort_values('product_id', ascending=True, inplace=True)

# df_pred_ungroup[df_pred_ungroup['group_index'] == 281]

In [25]:
df_pred_fut = df_fut[['product_id', 'tn_futuro_real', 'tn_prediccion_real']].groupby('product_id', as_index=False).sum()
#df_pred_fut['error'] = -(df_pred_fut['tn_futuro_real'] - df_pred_fut['tn_prediccion_real'])/df_pred_fut['tn_futuro_real']*100
#df_pred_fut['error_abs'] = -(df_pred_fut['tn_futuro_real'] - df_pred_fut['tn_prediccion_real'])
#df_pred_fut['weight'] = df_pred_fut['tn_futuro_real']/df_pred_fut['tn_futuro_real'].sum()
#df_pred_fut.sort_values('weight', ascending=False, inplace=True)

print(f"Primeras filas de futuro:\n{df_pred_fut.iloc[:11,:4]}")

Primeras filas de futuro:
    product_id  tn_futuro_real  tn_prediccion_real
0        20001             0.0         1177.233562
1        20002             0.0          783.391667
2        20003             0.0          819.136865
3        20004             0.0          550.266450
4        20005             0.0          581.811943
5        20006             0.0          456.547889
6        20007             0.0          517.314707
7        20008             0.0          365.795441
8        20009             0.0          439.505797
9        20010             0.0          416.244682
10       20011             0.0          483.263531


In [26]:
# df_fut_ungroup = df_fut[['group_index', 'tn_prediccion_real']]
# df_fut_ungroup = df_fut_ungroup.merge(prod_data_ungroup, on='group_index', how='left')
# df_fut_ungroup = df_fut_ungroup[['group_index', 'product_id', 'tn_prediccion_real', 'proporcion']]

# df_fut_ungroup = df_fut_ungroup[df_fut_ungroup['product_id'].isin(prod_a_predecir['product_id'])]
# df_fut_ungroup['prediccion_final'] = df_fut_ungroup['tn_prediccion_real'] * df_fut_ungroup['proporcion']
# df_fut_ungroup.sort_values('product_id', ascending=True, inplace=True)

In [27]:
prod_a_predecir = pd.read_csv('../productos_a_predecir.txt', sep='\t')

In [28]:
kaggle =  df_pred_fut[['product_id','tn_prediccion_real']]
kaggle['tn_prediccion_real'] = kaggle['tn_prediccion_real'].clip(lower=0)
kaggle.columns = ['product_id', 'tn']
kaggle = kaggle.merge(prod_a_predecir, on='product_id', how='right')
kaggle['tn'] = kaggle['tn'].fillna(0)

In [34]:
total = kaggle['tn'].sum()
correccion = total/30600
kaggle['tn'] = kaggle['tn']/correccion
print(total)

26946.56651985109


In [35]:
kaggle.to_csv(f"{folder}/{datetime.now().strftime('%Y%m%d_%H%M%S')}_{path_kaggle}", index=False)

In [36]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}\n\n\n")

---------------------------------------06_evaluar_prediccion----------------------------------------
----------------------------------------------FINALIZA----------------------------------------------



